In [51]:
! pip install monai==1.3.0


In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


In [53]:
import torch
import torch.nn as nn
import torch.optim as optim


In [54]:
!pip install torchvision


In [55]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [56]:
origa_images_folder = "ORIGA/ORIGA/Images"      # folder with all ORIGA images (.jpg)
     



In [57]:
origa_mat_file = "ORIGA/ORIGA/OrigaList.mat"



In [58]:
import os

print(os.listdir("ORIGA"))



['ORIGA']


In [59]:
import scipy.io

origa_mat_file = "ORIGA/ORIGA/OrigaList.mat"
mat = scipy.io.loadmat(origa_mat_file)

print(mat.keys())


dict_keys(['__header__', '__version__', '__globals__', 'Origa'])


In [60]:

origa = mat['Origa']
print(type(origa))          # Usually <class 'numpy.ndarray'>
print(origa.dtype)          # To see the structured array fields
print(origa.shape)          # Usually (1, 1) or similar



<class 'numpy.ndarray'>
[('Eye', 'O'), ('Filename', 'O'), ('ExpCDR', 'O'), ('Set', 'O'), ('Glaucoma', 'O')]
(1, 650)


In [61]:
print(origa.dtype.names) 

('Eye', 'Filename', 'ExpCDR', 'Set', 'Glaucoma')


In [62]:
filenames = origa['Filename'][0, 0]     # (N, 1) array of strings
labels = origa['Glaucoma'][0, 0]        # (N, 1) array of 0/1


In [63]:
filenames = [str(f[0]) for f in filenames]

In [64]:
# Convert labels to integers
labels = labels.flatten().astype(int).tolist()

In [66]:
import os

# Define full path to images
image_dir = "E:/Glucoma detection/U-Net/ORIGA/Images"
image_paths = [os.path.join(image_dir, fname + ".jpg") for fname in filenames]


In [67]:
from torch.utils.data import Dataset
from PIL import Image
import os

class ORIGADataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")  # convert to grayscale for LeNet
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [69]:
mat = scipy.io.loadmat("ORIGA/ORIGA/OrigaList.mat")
print(mat['Origa'].dtype)
print(mat['Origa'].shape)


[('Eye', 'O'), ('Filename', 'O'), ('ExpCDR', 'O'), ('Set', 'O'), ('Glaucoma', 'O')]
(1, 650)


In [70]:
origa_data = mat['Origa']
num_samples = origa_data.shape[1]

filenames = []
labels = []

# From your earlier loop
for i in range(num_samples):
    sample = origa_data[0, i]
    raw_name = sample[1][0]  # This is where it might be breaking

    print(f"Raw filename from .mat: '{raw_name}'")  # Debug line
    break




Raw filename from .mat: '001.jpg'


In [71]:
image_paths = []
final_labels = []

for i in range(num_samples):
    sample = origa_data[0, i]
    raw_name = sample[1][0].strip()  # This is '001.jpg'
    glaucoma = sample[4][0][0]       # This is 0 or 1

    # Construct the full path directly
    full_path = os.path.join(image_dir, raw_name)

    if os.path.exists(full_path):
        image_paths.append(full_path)
        final_labels.append(glaucoma)
    else:
        print(f"Missing image: {full_path}")


In [72]:
print("Total matched images:", len(image_paths))
print("First 5:", image_paths[:5], final_labels[:5])


Total matched images: 650
First 5: ['ORIGA\\ORIGA\\Images\\001.jpg', 'ORIGA\\ORIGA\\Images\\002.jpg', 'ORIGA\\ORIGA\\Images\\003.jpg', 'ORIGA\\ORIGA\\Images\\004.jpg', 'ORIGA\\ORIGA\\Images\\005.jpg'] [0, 0, 0, 0, 0]


In [73]:
# Now we go for preprocessing

In [74]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class OrigaDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        image = Image.open(img_path).convert('RGB')  # convert grayscale to RGB by repeating channels
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transforms for ResNet input
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean
                         [0.229, 0.224, 0.225])  # ImageNet std
])


In [81]:
dataset = OrigaDataset(image_paths, final_labels, transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Move inputs and labels in training loop:
inputs, labels = inputs.to(device), labels.to(device)



In [87]:



import torch.nn as nn
import torchvision.models as models
from torchvision.models import resnet18
model = resnet18(pretrained=True)
# Load pretrained ResNet18


# Freeze early layers (optional)
for param in model.parameters():
    param.requires_grad = True  # Fine-tune all layers


# Replace the final fully connected layer for 2 classes (glaucoma vs normal)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # binary classification

# Unfreeze the new fc layer parameters
for param in model.fc.parameters():
    param.requires_grad = True


In [88]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [90]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


Epoch 1/10, Loss: 0.5983
Epoch 2/10, Loss: 0.5482
Epoch 3/10, Loss: 0.5181
Epoch 4/10, Loss: 0.5072
Epoch 5/10, Loss: 0.5039
Epoch 6/10, Loss: 0.4609
Epoch 7/10, Loss: 0.4689
Epoch 8/10, Loss: 0.5305
Epoch 9/10, Loss: 0.4741
Epoch 10/10, Loss: 0.4590


In [91]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 76.15%


# ROI Extraction


In [93]:
import cv2
import numpy as np
from PIL import Image

def extract_roi(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Use Gaussian blur to smooth and threshold
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    _, thresh = cv2.threshold(blurred, 15, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Assume largest contour is optic disc
    c = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    
    roi = img[y:y+h, x:x+w]
    roi_resized = cv2.resize(roi, (32, 32))  # Resize for ResNet
    return Image.fromarray(cv2.cvtColor(roi_resized, cv2.COLOR_BGR2RGB))




ModuleNotFoundError: No module named 'cv2'